# Segmental retention models for representing the hydraulic properties of evolving structured soils

This notebook recreates Fig. 3.

In [ ]:
import segmental_models as svg
import vangenuchten as vg
import pandas as pd

import numpy as np
from scipy.constants import micro, milli, nano, centi
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
threshold_h = np.array([1, 0.3])

posd_h = np.concatenate((threshold_h, [0]))

Global settings for plots

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

h = np.logspace(-2.5, 3.5)

# This is the h over which conductivity will be integrated. It must be a larger range than what is plotted.
# Specifically, the high end must be high enough so that dmc/dh goes pretty much to zero *for the (de)compaction that
# you're making*. The low end should be the same as for h
h_int = np.logspace(-2.5, 5)

plotlim_h = np.flatnonzero(h_int > np.amax(h))[0]

## Decompaction

In [ ]:
soil = {'n': 1.42, 'a': 0.0054 / centi, 'mcr': 0.094, 'mcs': 0.418, 'Ks': 421.3 * milli / 60**2}

M = soil['Ks'] / soil['a'] ** 2 / (soil['mcs'] - soil['mcr']) ** 2.5

# Initial Van Genuchten retention and conductivity. Dimensions: [h]
VG_mc = vg.mc(h, **soil)
VG_K = (
    soil['Ks'] * vg.Kr(vg.S(h_int[None, :], **soil), **soil)[0]
)  # [0] removes the soil type dimension we're not using

# Retention curve segments [soil type, class, h]
sVG_comp = svg.comps(h, threshold_h, **soil)

# Retention sub-curves [class, h]
sVG_mc = svg.mc(sVG_comp, threshold_h, **soil)[0]  # [0] as above

In [ ]:
posd = pd.read_csv('MJGCB2020_PoSD.csv', index_col=0, comment='#')

posd['Micropores'] -= soil['mcr']
posd = posd[['Micropores', 'Mesopores', 'Macropores']].to_numpy()

# Weights or (de)compaction factors [year, component (micro, meso, macro)]
w = posd / sVG_mc[None, ..., 0]  # sVG_mc[..., 0] are the initial saturated water contents of the segments

In [ ]:
# (De)compacted retention curves [year, h]
sVG_mc_decomp = (sVG_mc[None, ...] * w[..., None]).sum(axis=-2)

In [ ]:
# (De)compacted conductivity curves [year, h]
sVG_K_decomp = (svg.dmcdh_int(h_int, threshold_h, **soil)[None, ...] * w[..., None]).sum(axis=-2) ** 2

sVG_K_decomp *= (
    M
    * (svg.mc(svg.comps(h_int[1:], threshold_h, **soil), threshold_h, **soil)[0, None, ...] * w[..., None]).sum(axis=-2)
    ** 0.5
)

# The (0, None,) part of the slice of above doesn't do anything here, but it signifies an intent: The 0 removes the soil
# type dimension we're not using (it has length 1), and None adds a year dimension we didn't have

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=sVG_mc_decomp.shape[0], figsize=(11, 6.75), sharex=True, sharey='row')

axes[0, 0].set_xscale('symlog', linthresh=0.01)
axes[1, 0].set_yscale('log')

for j, ax in enumerate(axes[0]):
    ax.plot(h, VG_mc, 'k', label='original retention / conductivity')

    if j < 2:
        ax.plot(posd_h, soil['mcr'] + np.cumsum(posd[j]), 'o', color=colors[1], label='measured porosity', zorder=5)
    else:
        ax.plot(
            posd_h,
            soil['mcr'] + np.cumsum(posd[j]),
            'o',
            color=colors[1],
            markerfacecolor='none',
            markeredgewidth=1.5,
            zorder=5,
        )

    ax.plot(h, soil['mcr'] + sVG_mc_decomp[j], color=colors[1], label='estimated ret. / cond.')
    ax.grid(alpha=0.25, which= 'both')

for j, ax in enumerate(axes[1]):
    ax.plot(h_int[:plotlim_h], VG_K[:plotlim_h], 'k', label='original')
    ax.plot(h_int[:plotlim_h], sVG_K_decomp[j, :plotlim_h], color=colors[1], label='estimated')
    ax.grid(alpha=0.25, which= 'both')

fig.supxlabel('pressure head $h$ [m]', y= .025)

for axo, title in zip(axes[0], ['initial', 'compacted', 'after 1 year', 'after 2 years']):
    axo.set_title(title)

axes[0, 0].add_line(  # We will generate legend from the [0, 0] subplot, so we need to add an entry for empty circles,
    #  which are not in that subplot
    Line2D(
        [],
        [],
        color=colors[1],
        marker='o',
        linestyle='None',
        markerfacecolor='none',
        markeredgewidth=1.5,
        label='simulated porosity',
    )
)
axes[0, 0].legend(loc='upper center', ncol=4, bbox_to_anchor=(2.1, -0.035))

axes[0, 0].invert_xaxis()
axes[0, 0].set_ylabel(r'water content $\theta$ [-]')
axes[1, 0].set_ylabel(r'hydr. conductivity $K$ [m/s]')

plt.subplots_adjust(wspace=0.1, hspace=0.22)

if 'xticklabels' in locals():
    axes[1,0].set_xticks(xticks[::2], xticklabels[::2])

    plt.savefig('MJGCB.pdf', bbox_inches="tight", pad_inches=.01)

In [ ]:
# Add minus signs to tick labels (replot after running this cell)
xticklabels = axes[1,0].get_xticklabels()
xticks = axes[1,0].get_xticks()

for Text in xticklabels:
    Text.set_text(Text.get_text().replace('10','\N{MINUS SIGN}10'))